<a href="https://colab.research.google.com/github/OldmanHades/PythonInspector/blob/main/OutputforPythonInspector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Original Model and Article for Review](https://machinelearningmastery.com/developing-a-python-program-using-inspection-tools/)<break>

Trying to Understand How this Python Inspector Works...

In [2]:

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

# Load MNIST training data
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train = torchvision.datasets.MNIST('./datafiles/', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)

# LeNet5 model
torch_model = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=(5,5), stride=1, padding=2),
    nn.Tanh(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
    nn.Tanh(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0),
    nn.Tanh(),
    nn.Flatten(),
    nn.Linear(120, 84),
    nn.Tanh(),
    nn.Linear(84, 10),
    nn.Softmax(dim=1)
)

# Training loop
def training_loop(model, optimizer, loss_fn, train_loader, n_epochs=100):
    model.train()
    for epoch in range(n_epochs):
        for data, target in train_loader:
            output = model(data)
            loss = loss_fn(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    model.eval()

# Run training
optimizer = optim.Adam(torch_model.parameters())
loss_fn = nn.CrossEntropyLoss()
training_loop(torch_model, optimizer, loss_fn, train_loader, n_epochs=20)

# Save model
torch.save(torch_model, "lenet5.pt")

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./datafiles/MNIST/raw/train-images-idx3-ubyte.gz to ./datafiles/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./datafiles/MNIST/raw/train-labels-idx1-ubyte.gz to ./datafiles/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./datafiles/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datafiles/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./datafiles/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datafiles/MNIST/raw



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, AveragePooling2D, Flatten
from tensorflow.keras.datasets import mnist

# LeNet5 model
keras_model = Sequential([
    Conv2D(6, (5,5), input_shape=(28,28,1), padding="same", activation="tanh"),
    AveragePooling2D((2,2), strides=2),
    Conv2D(16, (5,5), activation="tanh"),
    AveragePooling2D((2,2), strides=2),
    Conv2D(120, (5,5), activation="tanh"),
    Flatten(),
    Dense(84, activation="tanh"),
    Dense(10, activation="softmax")
])

# Reshape data to shape of (n_sample, height, width, n_channel)
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = np.expand_dims(X_train, axis=3).astype('float32')

# Train
keras_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
keras_model.fit(X_train, y_train, epochs=20, batch_size=32)

# Save
keras_model.save("lenet5.h5")

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/20
1875/1875 [==============================] - 41s 22ms/step - loss: 0.1501 - accuracy: 0.9552
Epoch 2/20
1875/1875 [==============================] - 40s 22ms/step - loss: 0.0684 - accuracy: 0.9787
Epoch 3/20
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0497 - accuracy: 0.9843
Epoch 4/20
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0420 - accuracy: 0.9866
Epoch 5/20
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0343 - accuracy: 0.9890
Epoch 6/20
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0325 - accuracy: 0.9895
Epoch 7/20
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0249 - accuracy: 0.9922
Epoch 8/20
1875/1875 [==============================] - 44s 23ms/step - loss: 0.0233 - accuracy: 0.9924
Epoch 9/20
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0201 - accuracy: 0.9933

In [ ]:
import torch
import tensorflow as tf

# Load the models
torch_model = torch.load("lenet5.pt")
keras_model = tf.keras.models.load_model("lenet5.h5")

# Extract weights from Keras model
keras_weights = keras_model.get_weights()

# Transform shape from Keras to PyTorch
for idx in [0, 2, 4]:
    # conv layers: (out, in, height, width)
    keras_weights[idx] = keras_weights[idx].transpose([3, 2, 0, 1])
for idx in [6, 8]:
    # dense layers: (out, in)
    keras_weights[idx] = keras_weights[idx].transpose()

# Set weights
torch_states = torch_model.state_dict()
for key, weight in zip(torch_states.keys(), keras_weights):
    torch_states[key] = torch.tensor(weight)
torch_model.load_state_dict(torch_states)

# Save new model
torch.save(torch_model, "lenet5-keras.pt")

In [ ]:
import numpy as np
import torch
import tensorflow as tf

# Load the models
torch_orig_model = torch.load("lenet5.pt")
keras_orig_model = tf.keras.models.load_model("lenet5.h5")
torch_converted_model = torch.load("lenet5-keras.pt")
keras_converted_model = tf.keras.models.load_model("lenet5-torch.h5")

# Create a random input
sample = np.random.random((28,28))

# Convert sample to torch input shape
torch_sample = torch.Tensor(sample.reshape(1,1,28,28))

# Convert sample to keras input shape
keras_sample = sample.reshape(1,28,28,1)

# Check output
keras_converted_output = keras_converted_model.predict(keras_sample, verbose=0)
keras_orig_output = keras_orig_model.predict(keras_sample, verbose=0)
torch_converted_output = torch_converted_model(torch_sample).detach().numpy()
torch_orig_output = torch_orig_model(torch_sample).detach().numpy()

np.set_printoptions(precision=4)
print(keras_orig_output)
print(torch_converted_output)
print()
print(torch_orig_output)
print(keras_converted_output)